In [1]:
%load_ext dotenv
%dotenv

In [2]:
import logging
import sys
import os

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

## 1. Preparation
### 1.1 Prepare LLM

In [3]:
from llama_index.llms import Ollama

OLLAMA_HOST = 'localhost'
OLLAMA_MODEL = 'mistral'
llm = Ollama(model=OLLAMA_MODEL, base_url="http://"+OLLAMA_HOST+":11434")

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, 
    # To save costs, we use a local model.
    # This will use a well-performing and fast default from Hugging Face.
    # this model has dim of 384 https://huggingface.co/BAAI/bge-small-en
    embed_model="local:BAAI/bge-small-en",
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 Prepare Graph Store

You need a running Nebula instance, you can start one with the Docker desktop Nebula Extension.
Once you have Nebula running, you have a first-time setup 
```cypher
ADD HOSTS "storaged0":9779,"storaged1":9779,"storaged2":9779
```
then you need to create the index before using it
```cypher
CREATE SPACE wikipedia(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
```
and
```cypher
USE wikipedia;
CREATE TAG entity(name string);
CREATE EDGE relationship(relationship string);
CREATE TAG INDEX entity_index ON entity(name(256));
```

In [5]:
assert os.environ["NEBULA_USER"] != ""
assert os.environ["NEBULA_PASSWORD"] != ""
assert os.environ["NEBULA_ADDRESS"] != ""

In [6]:
from llama_index.graph_stores import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name="wikipedia",
    edge_types=["relationship"],
    rel_prop_names=["relationship"],
    tags=["entity"],
)

In [7]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 2. Build the Knowledge Graph
### 2.1 Preprocess Data

In [8]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

### 2.2 Extract Triplets and Save to Graph
reference:
- [KnowledgeGraphIndex](https://docs.llamaindex.ai/en/stable/api_reference/indices/kg.html#llama_index.indices.knowledge_graph.KnowledgeGraphIndex)

In [16]:
from llama_index import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=5,
    include_embeddings=True,
    show_progress=True,
    # max_object_length: int = 128,
    space_name="wikipedia",
    edge_types=["relationship"],
    rel_prop_names=["relationship"],
    tags=["entity"],
    # to extract triplets, kg_triplet_extract_fn is used if not None,
    # kg_triplet_extract_fn: Optional[Callable] = None, 
    # else, the LLM from the service context is used with the kg_triple_extract_template if not None else the default triplet extract prompt
    # kg_triple_extract_template: Optional[BasePromptTemplate] = None,
)

Processing nodes:   0%|                                                                                                                                               | 0/16 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:   6%|████████▍                                                                                                                              | 1/16 [00:07<01:52,  7.50s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:  12%|████████████████▉                                                                                                                      | 2/16 [00:19<02:22, 10.19s/it]

...


Processing nodes:  88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                | 14/16 [02:17<00:19,  9.64s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:  94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 15/16 [02:32<00:11, 11.24s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Generating embeddings: 0it [00:00, ?it/s]
Processing nodes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [02:40<00:00, 10.06s/it]


### Conclusions from building the KG

- we can visualize the graph in Neo4j Bloom directly on top of Neo4j local or Neo4j AuraDB
- we have 1 Node type: Entity
    ```cypher
    MATCH (n)
    RETURN distinct labels(n)[0] as label, count(n) as node_count
    ```
- 80 Relationship type
    ```cypher
    MATCH p=()-->() RETURN count(p)
    ```
- Entities only have 1 field called `id`, we don't have entity type like "Person" etc... NER would be needed


## 5. Prepare for different query approaches

### 5.1 Graph RAG query engine

There are issues with the llama_index docs as I describe [here](https://github.com/run-llama/llama_index/issues/10474)

reference:

- [KGTableRetriever](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/kg.html#llama_index.indices.knowledge_graph.retrievers.KGTableRetriever) ??
- [KnowledgeGraphRAGRetriever](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/kg.html#llama_index.indices.knowledge_graph.retrievers.KnowledgeGraphRAGRetriever) "Retriever that perform SubGraph RAG towards knowledge graph."
- [RetrieverQueryEngine](https://docs.llamaindex.ai/en/stable/api_reference/query/query_engines/retriever_query_engine.html)

Explanation on the different Retrievers in the [docs](https://docs.llamaindex.ai/en/stable/examples/query_engine/knowledge_graph_rag_query_engine.html#why-knowledge-graph-rag-query-engine)

In [12]:
# in case we just built the index and we have it available
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False, 
    retriever_mode="keyword",
    response_mode="tree_summarize"
)

In [13]:
from llama_index.indices.knowledge_graph.retrievers import KGTableRetriever

assert type(kg_rag_query_engine.retriever) == KGTableRetriever

In [17]:
kg_rag__embed_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

In [21]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

kg_rag_fancy_retriever_query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

assert type(kg_rag_fancy_retriever_query_engine.retriever) == KnowledgeGraphRAGRetriever

## 6. Query with the engines

In [14]:
from IPython.display import Markdown, display

In [15]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill, also known as Star-Lord, is an entity who has been involved in several relationships. One of these relationships is described as "Speaks uncensored" with an object referred to as "Fuck in vol. 3". Another relationship is labeled as "Reunites with" and connects him to an entity named "Grandfather on earth".</b>

In [18]:
response_graph_rag = kg_rag__embed_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 032c7ecb-a63f-4ead-a22f-5a605141e376: == Plot ==
At their new headquarters on Knowhere, the Guardians of the Galaxy...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 16f10ef6-3cbb-4e95-aa5a-69fdd2ccd244: === Filming ===
Principal photography began on November 8, 2021, at Trilith S...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: dd662463-9d50-4be7-b44f-a17cde4cb032: == Cast ==
Chris Pratt as Peter Quill / Star-Lord:The half-human, half-Celest...
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill, also known as Star-Lord, is a half-human, half-Celestial character who was abducted from Earth as a child and raised by a group of alien thieves and smugglers called the Ravagers. He currently serves as the leader of the Guardians of the Galaxy. In the film, Quill is depicted in a "state of depression" due to the appearance of an alternate version of his dead lover Gamora who does not share the same affection for him. The role of Peter Quill is played by Chris Pratt. Quill speaks uncensored profanity, specifically the word 'fuck', in the film. Vin Diesel voices the character Groot, who is a member and accomplice of the Guardians. Quill decides to leave the Guardians and reunites with his grandfather on Earth.</b>

In [22]:
response_graph_rag = kg_rag_fancy_retriever_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill is an individual involved in various film projects. He has been identified as speaking uncensored language in several films, including "Film vol. 3" and the "Guardians of the galaxy holiday special." In relation to "Film vol. 3," Quill serves multiple roles such as cinematographer for Henry Braham and costume designer for Judianna Makovsky. Additionally, Quill decided to leave his guardians and reunited with his grandfather on earth. Industrial light & magic's stagecraft virtual production technology was utilized in the filming of "Film vol. 3." Chukwudi Iwuji is also part of this project as revealed by Quill.</b>